In [2]:
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
import json
import torch
from torch.utils.data import Dataset
import os
from datetime import datetime

class KnowledgeBaseDataset(Dataset):
    def __init__(self, knowledge_base, tokenizer, max_length=512):
        self.examples = []
        self.tokenizer = tokenizer
        self.max_length = max_length
        
        # Processa a base de conhecimento em formato de exemplos de treino
        for category, qa_pairs in knowledge_base["perguntas_respostas"].items():
            if isinstance(qa_pairs, dict):
                # Formata cada par de pergunta/resposta como um exemplo de treino
                question = qa_pairs.get("pergunta", "")
                answer = self._format_answer(qa_pairs.get("resposta", {}))
                
                if question and answer:
                    self.examples.append(self._format_example(question, answer))
                
                # Se houver exemplos de código, adiciona como exemplos separados
                if "exemplos" in qa_pairs:
                    for context, code in qa_pairs["exemplos"].items():
                        question = f"Mostre um exemplo de código para {context}"
                        self.examples.append(self._format_example(question, code))

    def _format_answer(self, answer):
        if isinstance(answer, dict):
            # Formata respostas estruturadas
            formatted = []
            for key, value in answer.items():
                if isinstance(value, list):
                    formatted.append(f"{key}:\n" + "\n".join(f"- {item}" for item in value))
                else:
                    formatted.append(f"{key}: {value}")
            return "\n".join(formatted)
        return str(answer)

    def _format_example(self, question, answer):
        # Formato do prompt: pergunta e resposta com marcadores específicos
        prompt = f"""###Pergunta
{question}

###Resposta
{answer}

###Fim"""
        return prompt

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        # Tokeniza o exemplo com padding e truncamento
        encodings = self.tokenizer(
            self.examples[idx],
            truncation=True,
            max_length=self.max_length,
            padding="max_length",
            return_tensors="pt"
        )
        
        return {
            "input_ids": encodings["input_ids"].squeeze(),
            "attention_mask": encodings["attention_mask"].squeeze()
        }

class KnowledgeManager:
    def __init__(self, base_path="knowledge_base"):
        self.base_path = base_path
        self.knowledge_base = self._load_knowledge()
        os.makedirs(base_path, exist_ok=True)

    def _load_knowledge(self):
        # Carrega todos os arquivos de conhecimento
        knowledge = {"perguntas_respostas": {}}
        for filename in os.listdir(self.base_path):
            if filename.endswith('.json'):
                with open(os.path.join(self.base_path, filename), 'r') as f:
                    domain_knowledge = json.load(f)
                    knowledge["perguntas_respostas"].update(domain_knowledge["perguntas_respostas"])
        return knowledge

    def add_knowledge(self, domain_name, new_knowledge):
        # Adiciona novo conhecimento e salva em arquivo separado
        filename = f"{domain_name}_{datetime.now().strftime('%Y%m%d')}.json"
        filepath = os.path.join(self.base_path, filename)
        
        with open(filepath, 'w') as f:
            json.dump(new_knowledge, f, indent=2)
        
        # Atualiza a base de conhecimento em memória
        self.knowledge_base["perguntas_respostas"].update(new_knowledge["perguntas_respostas"])

def train_model(model_name, knowledge_base, output_dir="trained_model"):
    """Treina ou continua o treinamento do modelo com nova base de conhecimento."""
    # Carrega modelo e tokenizer
    model = AutoModelForCausalLM.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    # Define o pad_token
    tokenizer.pad_token = tokenizer.eos_token
    
    # Prepara dataset
    dataset = KnowledgeBaseDataset(knowledge_base, tokenizer)
    
    # Configurações de treinamento
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=3,
        per_device_train_batch_size=4,
        save_steps=100,
        save_total_limit=2,
        logging_dir=f"{output_dir}/logs",
        learning_rate=2e-5,
    )
    
    # Inicializa trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
    )
    
    # Treina o modelo
    trainer.train()
    
    # Salva o modelo e tokenizer
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    
    return model, tokenizer


# Exemplo de uso
def main():
    # Inicializa o gerenciador de conhecimento
    knowledge_manager = KnowledgeManager()
    
    # Adiciona conhecimento inicial (estagiário)
    calculo_knowledge = {
        "perguntas_respostas": {
            "geral": {
                "pergunta": "O que é o objeto estagiario e qual sua função principal?",
                "resposta": "O objeto estagiario retorna os dados da matrícula..."
            }
            # ... resto do conhecimento do estagiário
        }
    }
    knowledge_manager.add_knowledge("calculo", calculo_knowledge)
    
    # Mais tarde, adiciona conhecimento sobre cálculo
    estagiario_knowledge = {
    "perguntas_respostas": {
        "geral": {
        "pergunta": "O que é o objeto estagiario e qual sua função principal?",
        "resposta": "O objeto estagiario retorna os dados da matrícula que sejam do tipo estagiário no cálculo. Ele permite acessar todas as informações relacionadas ao cadastro e configurações do estagiário."
        },
        "dados_pagamento": {
        "pergunta": "Como posso acessar as informações bancárias e de pagamento do estagiário?",
        "resposta": {
            "descricao": "Existem várias propriedades relacionadas ao pagamento:",
            "variaveis": [
            "formaPagamento: Retorna a forma de pagamento do estagiário",
            "bancoPagamento: Retorna o banco de pagamento",
            "agenciaPagamento: Retorna a agência de pagamento",
            "bolsaEstudos: Retorna o valor da bolsa de estudos (remuneração)"
            ],
            "exemplo": {
            "forma_pagamento": "if (estagiario.formaPagamento == FormaPagamento.DINHEIRO) { imprimir 'Forma de pagamento é em dinheiro'; }",
            "dados_bancarios": "String bancoPagamento = estagiario.bancoPagamento;\nString agenciaPagamento = estagiario.agenciaPagamento;"
            }
        }
        },
        "dados_salariais": {
        "pergunta": "Como posso obter informações sobre o plano salarial do estagiário?",
        "resposta": {
            "descricao": "O objeto estagiario possui várias propriedades relacionadas à estrutura salarial:",
            "variaveis": [
            "grupoFuncional: Retorna o grupo funcional",
            "planoSalarial: Retorna o plano salarial",
            "classeSalarial: Retorna a classe salarial",
            "nivelSalarial: Retorna o nível salarial",
            "nivelSalarialCoeficiente: Indica se o nível salarial possui coeficiente"
            ],
            "exemplos": {
            "acesso_basico": "String planoSalarial = estagiario.planoSalarial;\nString classeSalarial = estagiario.classeSalarial;",
            "verificacao_coeficiente": "Boolean nivelSalarialCoeficiente = estagiario.nivelSalarialCoeficiente;"
            }
        }
        },
        "periodo_estagio": {
        "pergunta": "Como verificar as datas e períodos relacionados ao estágio?",
        "resposta": {
            "descricao": "Existem várias propriedades para controle de datas do estágio:",
            "variaveis": [
            "dataInicioEstagio: Data de início do estágio",
            "dataFinalEstagio: Data final do estágio",
            "dataProrrogacao: Data da prorrogação do estágio"
            ],
            "exemplo": {
            "verificacao_data": "Date dataInicioEstagio = estagiario.dataInicioEstagio;\nDate dataFinalEstagio = estagiario.dataFinalEstagio;",
            "verificacao_prorrogacao": "if (estagiario.dataProrrogacao != null) { Date dataProrrogacao = estagiario.dataProrrogacao; }"
            }
        }
        },
        "ferias_beneficios": {
        "pergunta": "Como consultar informações sobre férias e benefícios do estagiário?",
        "resposta": {
            "descricao": "O objeto possui propriedades específicas para férias e benefícios:",
            "variaveis": [
            "recebeDecimoTerceiro: Indica se recebe 13º conforme configuração do cargo",
            "diasDireitoFerias: Dias de direito de férias conforme configuração",
            "mesesAquisicaoFerias: Meses para aquisição de férias"
            ],
            "exemplos": {
            "verificacao_13": "Boolean recebeDecimoTerceiro = estagiario.recebeDecimoTerceiro;",
            "consulta_ferias": "int diasDireitoFerias = estagiario.diasDireitoFerias;\nint mesesAquisicaoFerias = estagiario.mesesAquisicaoFerias;"
            }
        }
        },
        "carga_horaria": {
        "pergunta": "Como verificar a carga horária do estagiário?",
        "resposta": {
            "descricao": "Use a propriedade quantidadeHorasMes",
            "exemplo": "BigDecimal quantidadeHorasMes = estagiario.quantidadeHorasMes;\nif (quantidadeHorasMes < BigDecimal.valueOf(100)) {\n    imprimir 'Quantidade de horas mês é menor que 100';\n}",
            "observacao": "O valor retornado é do tipo BigDecimal para maior precisão no cálculo"
        }
        },
        "dados_organizacionais": {
        "pergunta": "Como obter informações sobre a localização organizacional do estagiário?",
        "resposta": {
            "descricao": "Use a propriedade descricaoOrganograma",
            "exemplo": "String descricaoOrganograma = estagiario.descricaoOrganograma;",
            "observacao": "Retorna a descrição completa do organograma onde o estagiário está alocado"
        }
        }
    }
    }
    knowledge_manager.add_knowledge("estagiario", estagiario_knowledge)
    
    # Treina o modelo com todo o conhecimento
    model_name = "meta-llama/Llama-3.2-3B-Instruct"
    model, tokenizer = train_model(model_name, knowledge_manager.knowledge_base)
    
    return model, tokenizer

if __name__ == "__main__":
    model, tokenizer = main()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

ValueError: The model did not return a loss from the inputs, only the following keys: logits,past_key_values. For reference, the inputs it received are input_ids,attention_mask.